# Programming Exercise 7: K-means Clustering and Principal Component Analysis

```
by Seokkyu Kong
Date: 2016-04-06
Summary: Coursera machine learning (Prof. Andrew Ng) 강의 내용과 assignment는 octave(matlab)으로 이루어진다. 
해당 코드를 python으로 구현해본다.

Andrew Ng 교수의 강의: https://www.coursera.org/learn/machine-learning/
```

Numpy 와 MATLAB 참고 자료

- [Numpy for Matlab users #1](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html)
- [NumPy for MATLAB users #2](http://mathesaurus.sourceforge.net/matlab-numpy.html)


## Introduction

이번 연습문제에서는, K-means 클러스터링 알고리즘을 구현하고 이미지를 압축하는데 응용해본다. 2번째 부분에서는 principal component analysis 주성분 분석을 구현해서 얼굴 이미지에 대한 낮은 차원의 표현을 찾게 된다.

### Files included in this exercise

- ex7.m - Octave/MATLAB script for the first exercise on K-means
- ex7 pca.m - Octave/MATLAB script for the second exercise on PCA
- ex7data1.mat - Example Dataset for PCA
- ex7data2.mat - Example Dataset for K-means
- ex7faces.mat - Faces Dataset
- bird small.png - Example Image
- displayData.m - Displays 2D data stored in a matrix
- drawLine.m - Draws a line over an exsiting figure
- plotDataPoints.m - Initialization for K-means centroids
- plotProgresskMeans.m - Plots each step of K-means as it proceeds
- runkMeans.m - Runs the K-means algorithm
- submit.m - Submission script that sends your solutions to our servers
- [*] pca.m - Perform principal component analysis
- [*] projectData.m - Projects a data set into a lower dimensional space
- [*] recoverData.m - Recovers the original data from the projection
- [*] findClosestCentroids.m- Find closest centroids (used in K-means)
- [*] computeCentroids.m - Compute centroid means (used in K-means)
- [*] kMeansInitCentroids.m - Initialization for K-means centroids

연습문제 첫번째 파트를 통해서, 스크립트 ex7.m을 사용할 것이고 두번째 파트에서는 ex7_pca.m을 사용하게 된다. 이들 스크립트는 문제에 대한 데이터셋을 준비하고 당신이 작성하게 될 함수를 호출한다.

## K-means Clustering

이번 연습문제에서, K-means 알고리즘을 구현하고 이미지 압축을 위해 그것을 사용할 것이다. 먼저 example 2D 데이터셋으로 시작하는데 K-means 알고리즘이 어떻게 동작하는지에 대한 직관을 얻을 수 있다. 그 이후에 K-means 알고리즘을 사용해서 이미지 압축을 하는데 그 이미지 내에 가장 많이 공통적으로 발생하는 컬러의 수를 감소시킴으로 가능하다. 여기서는 ex7.m 을 사용한다.

### 1.1 Implementing K-means

K-means 알고리즘은 자동적으로 유사한 데이터 examples를 함께 묶어 주는 방법이다. 구체적으로, training set {x(1),,, x(m)}이 주어졌을 때 (X^(i) ( R^n), 데이터를 몇개의 결합된 "clusters 군집" 으로 그룹핑하고 싶을 것이다. 

**K-means 에 대한 직관은 반복적인 절차인데, 초기 centroids 줌심점들을 추측하고 이 예측을 가다듬는데, 반복적으로 examples를 그들의 가장 가까운 중심점으로 할당하고 할당된 점들을 기준으로 중심점을 재계산하는 것이다.**

K-means 알고리즘은 다음처럼 동작한다.


**알고리즘의 내부 루프는 반복적으로 다음 2 단계를 수행한다. (i) 각각의 training example x(i)를 가장 가까운 중심점에 할당하면서 (ii) 중심점에 할당된 점들을 사용해서 각 중심점의 평균을 재계산한다.**

K-means 알고리즘은 항상 중심점에 대한 유한한 평균 집합으로 수렴하게 된다.

**수렴된 솔루션이 항상 이상적인 것은 아니고 초기에 설정한 중심점에 의존적임을 주목하라. 그래서 실제로 K-means 알고리즘은 항상 서로 다른 무작위 random 초기화로 여러번 실행한다.**

서로 다른 무작위 초기화에서 서로 다른 솔루션들 중 하나를 선택하는 방법은 가장 낮은 cost function value (distortion) 을 갖는 것을 선택하는 것이다.

당신은 K-means 알고리즘의 2단계를 다음 섹션에서 개별적으로 구현할 것이다.


#### 1.1.1 Finding closest centroids

K-means 알고리즘의 "군집 할당" 단계에서, 알고리즘은 각각의 training example x(i)를 가장 가까운 중심점에 할당한다. 특별히, 각 exaple i 에대해서 우리는 다음과 같이 설정한다.

여기서 c(i)는 x(i)에 가장 가까운 중심점의 index가 되고, mu(j)는 j번째 중심점의 위치(값)이 된다. **c(i)는 시작 코드에서 idx(i)에 일치함을 주목해라.**

당신은 findClosestCentroids.m에 있는 코드를 완성하면 된다. 이 함수는 데이터 행렬 X와 cetroids 에 있는 모든 중심점의 위치를 가지고 1차 배열 idx를 출력하는데 각 training example에 대해서 가장 가까운 중심점의 index를 유지한다. (index는 {1,,,K} 사이의 값인데 K는 전체 중심점의 갯수이다.)

findClosestCetroids.m 에 있는 코드를 완성한 이후에, 스크립트 ex7.m은 코드를 실행시키는데 출력값 [1 3 2]는 처음 3개의 examples에 대한 중심점 할당에 일치한다.


In [1]:
# You should now submit your solutions.

#### 1.1.2 Comuting centroid means

중심점에 할당된 모든 점들에 대해서 알고리즘의 2번째 단계는 각각의 중심점에 대해서 중심점에 할당된 점들의 평균을 재계산 한다. 특별히, 각 중심점 k에 대해서 우리는 다음과 같이 설정한다.

**여기서 Ck는 중심점 k에 할당된 examples의 집합이 된다.** 구체적으로 만약 2개의 examples가 말하자면 x(3), x(5)가 중심점 k = 2에 할당되었다면, mu2 = 1/2(x(3) + x(5)) 로 업데이트 해야 한다.

computeCentroids.m 에 있는 코드를 완성해야 한다. **중심점들에 대한 루프를 사용해서 이 함수를 구현할 수 있다. 또한 examples 에 대한 루프도 사용할 수 있는데, 가능하다면 그와 같은 루프 사용 없는 벡터화 구현을 사용해야 한다.** 그러면 코드는 더 빨리 실행될 것이다.

일단 computeCentroids.m 에 있는 코드를 완성하면 스크립트 ex7.m은 코드를 실행시키고 K-means 첫 단계 이후에 중심점을 출력한다.


In [2]:
# You should now submit your solutions.


### 1.2 K-means on example dataset

그림 1: The expected output

2개의 함수를 완성한 이후에(findClosestCentroids와 computeCentroids), ex7.m의 다음 단계는 toy 2D dataset 상에 K-means 알고리즘을 실행하는데, K-means 가 어떻게 동작하는지 이해하는데 도움이 될 것이다. **당신의 함수는 runKmeans.m 스크립트 내에서 호출된다. 우리는 그 함수가 어떻게 동작하는지 이해하기 위해 그 함수를 볼 것을 권장한다. 코드는 루프 내에서 당신이 구현한 2개의 함수를 호출함을 주목해라.**

다음 단계를 실행할 때, K-means 코드는 가시화 그래프를 생성하는데, 매 반복마다 알고리즘의 진행이 어떻게 이루어지는지 보여준다. enter를 여러번 누르면 K-means 알고리즘의 각 단계가 중심점과 군집 할당을 어떻게 변경시키는지 볼 수 있다. 끝에 가서, 당신의 그림은 그림 1에 보여진 것과 같이 보여야 한다.




### 1.3 Random initialization

